# metrics

> Metrics (e.g. ROC) and also a `predict_model` function that is more efficient than FastAI defaults when the dataset is small.

In [ ]:
#| default_exp metrics

In [ ]:
#| hide

from nbdev.showdoc import *

## Predictions given xval and yval:

In [ ]:
#| export
from fastai.vision.all import *
import torch
from statistics import mean,stdev
import numpy as np
import scikitplot 

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score,average_precision_score,precision_recall_curve,roc_curve,auc,classification_report,confusion_matrix
from scipy import interp

import seaborn as sns
import pandas as pd

In [ ]:
#| export

@torch.no_grad()
def predict_model(xval,yval,model,aug_pipelines_test,numavg=3,criterion = CrossEntropyLossFlat(),deterministic=False):
    "Note that this assumes xval is entire validation set. If it doesn't fit in memory, can't use this guy"
    
    model.eval()

    N=xval.shape[0]
    
    if not deterministic:

        probs=0
        for _ in range(numavg):

            probs += torch.softmax(model(aug_pipelines_test[0](xval)),dim=1) #test time augmentation. This also gets around issue of randomness in the dataloader in each session...

        probs *= 1/numavg
        
    else:
        probs = torch.softmax(model(xval),dim=1)

    
    ypred = cast(torch.argmax(probs, dim=1),TensorCategory)

    correct = (ypred == yval)#.type(torch.FloatTensor)

    #correct = (torch.argmax(ypred,dim=1) == yval).type(torch.FloatTensor)
    num_correct = correct.sum()
    accuracy = num_correct/N

    #val_loss = criterion(scores,yval)
    
    return probs,ypred,accuracy.item()#,val_loss.item()

In [ ]:
#| export

def predict_ensemble(yval,scores1,scores2):
    "scores can be normalized (softmax) or not"

    N=yval.shape[0]

    scores = 0.5*scores1 + 0.5*scores2

    ypred = cast(torch.argmax(scores, dim=1),TensorCategory)

    correct = (ypred == yval)#.type(torch.FloatTensor)

    #correct = (torch.argmax(ypred,dim=1) == yval).type(torch.FloatTensor)
    num_correct = correct.sum()
    accuracy = num_correct/N
    
    return ypred,accuracy.item()

In [ ]:
#| export

def classification_report_wrapper(ypred, y, int_to_classes, print_report=True):
    # Convert ypred and y to numpy arrays
    ypred = ypred.cpu().numpy()
    y = y.cpu().numpy()

    # Get the unique classes in y
    unique_y = np.unique(y)

    # Create labels from unique_y and vocab
    labels = [int_to_classes[i] for i in unique_y]

    # Get the classification report as a dictionary
    report = classification_report(y, ypred, target_names=labels, output_dict=True)

    if print_report:
        print(classification_report(y, ypred, target_names=labels))

    return report

In [ ]:
#| hide

ypred=np.array([0,1,0,1])
ypred2=np.array([0,0,0,0])
y=np.array([0,0,0,1])
int_to_classes={0:'Dog',1:'Cat'}
labels = ['Dog','Cat']
#classification_report_wrapper(ypred, y, int_to_classes)

report = classification_report(y, ypred, target_names=labels,output_dict=True)
report2 = classification_report(y, ypred2, target_names=labels,output_dict=True)
print(classification_report(y, ypred, target_names=labels))
print(classification_report(y, ypred2, target_names=labels))


              precision    recall  f1-score   support

         Dog       1.00      0.67      0.80         3
         Cat       0.50      1.00      0.67         1

    accuracy                           0.75         4
   macro avg       0.75      0.83      0.73         4
weighted avg       0.88      0.75      0.77         4

              precision    recall  f1-score   support

         Dog       0.75      1.00      0.86         3
         Cat       0.00      0.00      0.00         1

    accuracy                           0.75         4
   macro avg       0.38      0.50      0.43         4
weighted avg       0.56      0.75      0.64         4



/Users/hamishhaggerty/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hamishhaggerty/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hamishhaggerty/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

In [ ]:
#| export

#This code comes from https://gist.github.com/geblanco/5cfe4a3224e021113968a8c7ebe31419
def format_classification_report(data_dict):


    non_label_keys = ["accuracy", "macro avg", "weighted avg"]
    y_type = "binary"
    digits = 2

    target_names = [
        "%s" % key for key in data_dict.keys() if key not in non_label_keys
    ]

    # labelled micro average
    micro_is_accuracy = (y_type == "multiclass" or y_type == "binary")

    headers = ["precision", "recall", "f1-score", "support"]
    p = [data_dict[l][headers[0]] for l in target_names]
    r = [data_dict[l][headers[1]] for l in target_names]
    f1 = [data_dict[l][headers[2]] for l in target_names]
    s = [data_dict[l][headers[3]] for l in target_names]

    rows = zip(target_names, p, r, f1, s)

    if y_type.startswith("multilabel"):
        average_options = ("micro", "macro", "weighted", "samples")
    else:
        average_options = ("micro", "macro", "weighted")

    longest_last_line_heading = "weighted avg"
    name_width = max(len(cn) for cn in target_names)
    width = max(name_width, len(longest_last_line_heading), digits)
    head_fmt = "{:>{width}s} " + " {:>9}" * len(headers)
    report = head_fmt.format("", *headers, width=width)
    report += "\n\n"
    row_fmt = "{:>{width}s} " + " {:>9.{digits}f}" * 3 + " {:>9}\n"
    for row in rows:
        report += row_fmt.format(*row, width=width, digits=digits)
    report += "\n"

    # compute all applicable averages
    for average in average_options:
        if average.startswith("micro") and micro_is_accuracy:
            line_heading = "accuracy"
        else:
            line_heading = average + " avg"

        if line_heading == "accuracy":
            avg = [data_dict[line_heading], sum(s)]
            row_fmt_accuracy = "{:>{width}s} " + \
                    " {:>9.{digits}}" * 2 + " {:>9.{digits}f}" + \
                    " {:>9}\n"
            report += row_fmt_accuracy.format(line_heading, "", "",
                                              *avg, width=width,
                                              digits=digits)
        else:
            avg = list(data_dict[line_heading].values())
            report += row_fmt.format(line_heading, *avg,
                                     width=width, digits=digits)
    return report



In [ ]:
#| export

def Mean_Report(reports, classes):
    N = len(reports)
    mean_report = {}
    for k in reports[0].keys():
        if k!='accuracy':
            mean_report[k] = {}
            for metric in reports[0][k].keys():
                att = 0
                for _report in reports:
                    att += _report[k][metric]
                mean_report[k][metric] = att / N
        else:
            att = 0
            for _report in reports:
                att += _report[k]
                
            mean_report[k] = att / N
    return mean_report


In [ ]:
#| export

def Mean_Results(results,vocab):
    "Get mean classif report and display it"

    lst = list(vocab) + ['accuracy', 'macro avg', 'weighted avg']
    reports=[]
    accs=[]
    for i in results.keys():
        if type(i)!=int:
            continue
        report = {j:results[i][j] for j in results[i].keys() if j in lst}
        reports.append(report)
        accs.append(results[i]['acc'])
    mean_report = Mean_Report(reports,vocab)
    print(format_classification_report(mean_report))
    
    print(f'mean acc is {mean(accs)} with std {stdev(accs)}')

    return mean_report

In [ ]:
#| hide

# _mean_report = Mean_Report([report,report2],vocab=['Dog','Cat'])

# test_eq(_mean_report.keys(),report.keys())

In [ ]:
#| hide

# report['acc']=0.01
# report2['acc']=0.02
# Reports = {0:report,1:report2}
# Mean_Results(Reports,vocab=['Dog','Cat'])

In [ ]:
#| hide
#Verbose test to make sure it is working
# for k in report.keys():
    
#     if k!='accuracy':
        
#         for metric in report[k].keys():

            
#             diff = abs(_mean_report[k][metric] - 0.5*(report[k][metric]+report2[k][metric]))
#             #assert diff<1e-7
            
    
#     if k == 'accuracy':
        
#         diff = abs(_mean_report[k]-0.5*(report[k]+report2[k]))
#         #test_eq(_mean_report[k],0.5*(report[k]+report2[k]))
        
    
#     assert diff<1e-7

In [ ]:
#| hide

#print(format_classification_report(_mean_report))

In [ ]:
#| hide

#print(format_classification_report(report))

In [ ]:
#| hide

#print(format_classification_report(report2))

In [ ]:
#| export

def print_confusion_matrix(ypred, y, vocab):
    # Convert ypred and y to numpy arrays
    ypred = ypred.cpu().numpy()
    y = y.cpu().numpy()
    
    # Get the class labels from vocab
    labels = [vocab[i] for i in range(len(vocab))]
    
    # Create the confusion matrix
    cm = confusion_matrix(y, ypred)
    
    # Create a DataFrame from the confusion matrix
    df_cm = pd.DataFrame(cm, index = labels, columns = labels)
    
    # Use seaborn to create a heatmap of the confusion matrix with blue and white colors
    sns.heatmap(df_cm, annot=True, cmap="Blues")


We monkey patch some plotting functions from scikitplot and edit them - we need greater control of legend etc

In [ ]:
#| export

def _plot_precision_recall(y_true, y_probas,
                          title='Precision-Recall Curve',
                          plot_micro=True,
                          classes_to_plot=None, ax=None,
                          figsize=None, cmap='nipy_spectral',
                          title_fontsize="large",
                          text_fontsize="small"):
    """Generates the Precision Recall Curve from labels and probabilities. This is moneky patched from: 
    https://github.com/reiinakano/scikit-plot/blob/26007fbf9f05e915bd0f6acb86850b01b00944cf/scikitplot/metrics.py

    """
    y_true = np.array(y_true)
    y_probas = np.array(y_probas)

    classes = np.unique(y_true)
    probas = y_probas

    if classes_to_plot is None:
        classes_to_plot = classes

    binarized_y_true = label_binarize(y_true, classes=classes)
    if len(classes) == 2:
        binarized_y_true = np.hstack(
            (1 - binarized_y_true, binarized_y_true))

    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=figsize)

    ax.set_title(title, fontsize=title_fontsize)

    indices_to_plot = np.in1d(classes, classes_to_plot)
    for i, to_plot in enumerate(indices_to_plot):
        if to_plot:
            average_precision = average_precision_score(
                binarized_y_true[:, i],
                probas[:, i])
            precision, recall, _ = precision_recall_curve(
                y_true, probas[:, i], pos_label=classes[i])
            color = plt.cm.get_cmap(cmap)(float(i) / len(classes))
            ax.plot(recall, precision, lw=2,
                    label='{0} '
                          '(area = {1:0.3f})'.format(classes[i],
                                                     average_precision),
                    color=color)

    if plot_micro:
        precision, recall, _ = precision_recall_curve(
            binarized_y_true.ravel(), probas.ravel())
        average_precision = average_precision_score(binarized_y_true,
                                                    probas,
                                                    average='micro')
        ax.plot(recall, precision,
                label='micro-avg '
                      '(area = {0:0.3f})'.format(average_precision),
                color='navy', linestyle=':', linewidth=4)

    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.tick_params(labelsize=text_fontsize)
    ax.legend(loc='lower left', fontsize=text_fontsize)
    return ax

def _plot_roc(y_true, y_probas, title='ROC Curves',
                   plot_micro=True, plot_macro=True, classes_to_plot=None,
                   ax=None, figsize=None, cmap='nipy_spectral',
                   title_fontsize="large", text_fontsize="medium"):
    """Generates the ROC curves from labels and predicted scores/probabilities. Monkey patched like above function.


    """
    y_true = np.array(y_true)
    y_probas = np.array(y_probas)

    classes = np.unique(y_true)
    probas = y_probas

    if classes_to_plot is None:
        classes_to_plot = classes

    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=figsize)

    ax.set_title(title, fontsize=title_fontsize)

    fpr_dict = dict()
    tpr_dict = dict()

    indices_to_plot = np.in1d(classes, classes_to_plot)
    for i, to_plot in enumerate(indices_to_plot):
        fpr_dict[i], tpr_dict[i], _ = roc_curve(y_true, probas[:, i],
                                                pos_label=classes[i])
        if to_plot:
            roc_auc = auc(fpr_dict[i], tpr_dict[i])
            color = plt.cm.get_cmap(cmap)(float(i) / len(classes))
            ax.plot(fpr_dict[i], tpr_dict[i], lw=2, color=color,
                    label='{0} (area = {1:0.2f})'
                          ''.format(classes[i], roc_auc))

    if plot_micro:
        binarized_y_true = label_binarize(y_true, classes=classes)
        if len(classes) == 2:
            binarized_y_true = np.hstack(
                (1 - binarized_y_true, binarized_y_true))
        fpr, tpr, _ = roc_curve(binarized_y_true.ravel(), probas.ravel())
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr,
                label='micro-avg'
                      '(area = {0:0.2f})'.format(roc_auc),
                color='deeppink', linestyle=':', linewidth=4)

    if plot_macro:
        # Compute macro-average ROC curve and ROC area
        # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr_dict[x] for x in range(len(classes))]))

        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(len(classes)):
            mean_tpr += interp(all_fpr, fpr_dict[i], tpr_dict[i])

        # Finally average it and compute AUC
        mean_tpr /= len(classes)
        roc_auc = auc(all_fpr, mean_tpr)

        ax.plot(all_fpr, mean_tpr,
                label='macro-avg'
                      '(area = {0:0.2f})'.format(roc_auc),
                color='navy', linestyle=':', linewidth=4)

    ax.plot([0, 1], [0, 1], 'k--', lw=2)
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=text_fontsize)
    ax.set_ylabel('True Positive Rate', fontsize=text_fontsize)
    ax.tick_params(labelsize=text_fontsize)
    ax.legend(loc='lower right', fontsize=text_fontsize)
    return ax

#| export

def plot_roc(ytest,probs,int_to_classes):
    
    #We want the AUC dict; and we want a plot as well.
    
    ytest = ytest.cpu().numpy()
    _ytest = [int_to_classes[i] for i in ytest] #e.g. ['AK','MEL',...]

    #scikitplot.metrics.plot_roc(_ytest, probs,plot_micro=True,plot_macro=False)
    _plot_roc(_ytest, probs,plot_micro=True,plot_macro=False)

def plot_pr(ytest,probs,int_to_classes):
    
    #We want the AUC dict; and we want a plot as well.
    
    ytest = ytest.cpu().numpy()
    _ytest = [int_to_classes[i] for i in ytest] #e.g. ['AK','MEL',...]

    #scikitplot.metrics.plot_precision_recall(_ytest, probs)#,plot_micro=True,plot_macro=False)
    _plot_precision_recall(_ytest, probs,plot_micro=True)

    plt.legend(loc='best', fontsize='small')
    plt.show()

Functions to plot ROC curve and precision-recall curves:

In [ ]:
#| export

def plot_roc(ytest,probs,int_to_classes):
    
    #We want the AUC dict; and we want a plot as well.
    
    ytest = ytest.cpu().numpy()
    _ytest = [int_to_classes[i] for i in ytest] #e.g. ['AK','MEL',...]
    
    #scikitplot.metrics.plot_roc(_ytest, probs,plot_micro=True,plot_macro=False)
    _plot_roc(_ytest, probs,plot_micro=True,plot_macro=False)

def plot_pr(ytest,probs,int_to_classes):
    
    #We want the AUC dict; and we want a plot as well.
    
    ytest = ytest.cpu().numpy()
    _ytest = [int_to_classes[i] for i in ytest] #e.g. ['AK','MEL',...]
    
    #scikitplot.metrics.plot_precision_recall(_ytest, probs)#,plot_micro=True,plot_macro=False)
    _plot_precision_recall(_ytest, probs,plot_micro=True)

    plt.legend(loc='best', fontsize='small')
    plt.show()

In [ ]:
#| export

def Auc_Dict(ytest,probs,int_to_classes=None):
    "Mostly used to verify results of plot (debug)"

    # # define your ytest and probs variables
    # ytest = ['cat', 'dog', 'bear', 'dog', 'cat']
    # probs = np.array([[0.7, 0.2, 0.1], [0.3, 0.6, 0.1], [0.2, 0.3, 0.5], [0.1, 0.7, 0.2], [0.8, 0.1, 0.1]])

    # calculate AUC for each class
    auc_dict = {}
    for i, cls in enumerate(np.unique(ytest)):
        y_true = np.array([1 if y == cls else 0 for y in ytest]) # one-vs-all labels
        y_score = probs[:, i] # get probability for current class
        auc = roc_auc_score(y_true, y_score)#, multi_class='ovr')
        auc_dict[cls] = auc

    if int_to_classes!=None:
        auc_dict = {int_to_classes[i]:auc_dict[i] for i in auc_dict}
    
    return auc_dict


def Pr_Dict(ytest,probs,int_to_classes=None):
    "Mostly used to verify results of plot (debug)"

    # # define your ytest and probs variables
    # ytest = ['cat', 'dog', 'bear', 'dog', 'cat']
    # probs = np.array([[0.7, 0.2, 0.1], [0.3, 0.6, 0.1], [0.2, 0.3, 0.5], [0.1, 0.7, 0.2], [0.8, 0.1, 0.1]])

    # calculate AUC for each class
    pr_dict = {}
    for i, cls in enumerate(np.unique(ytest)):
        y_true = np.array([1 if y == cls else 0 for y in ytest]) # one-vs-all labels
        y_score = probs[:, i] # get probability for current class
        pr = average_precision_score(y_true, y_score)#, multi_class='ovr')
        pr_dict[cls] = pr

    if int_to_classes!=None:
        pr_dict = {int_to_classes[i]:pr_dict[i] for i in pr_dict}
    
    return pr_dict


In [ ]:
#| export


@torch.no_grad()
def predict_whole_model(dls_test, model, aug_pipelines_test, numavg=3, criterion=CrossEntropyLossFlat(), deterministic=False):
    """
    Predicts the labels and probabilities for the entire test set using the specified model and data augmentation
    pipelines. Returns a dictionary containing the labels, probabilities, predicted labels, and accuracy.

    Args:
        dls_test: The test dataloader.
        model: The trained model.
        aug_pipelines_test: The test data augmentation pipelines.
        numavg: The number of times to perform test-time augmentation.
        criterion: The loss function to use for computing the accuracy.
        deterministic: Whether to use deterministic computation.

    Returns:
        A dictionary containing the labels, probabilities, predicted labels, and accuracy.
    """
    model.eval()
    total_len = len(dls_test.dataset)
    y = torch.zeros(total_len, dtype=torch.long)
    probs = torch.zeros(total_len, model.head.out_features)
    ypred = torch.zeros(total_len, dtype=torch.long)

    start_idx = 0
    for xval, yval in dls_test.train:
        end_idx = start_idx + len(xval)
        _probs, _ypred, acc = predict_model(xval, yval, model, aug_pipelines_test, numavg, criterion, deterministic)
        y[start_idx:end_idx] = yval
        probs[start_idx:end_idx] = _probs
        ypred[start_idx:end_idx] = _ypred
        start_idx = end_idx

    # Calculate the overall accuracy
    acc = (ypred == y).float().mean().item()

    # Return the predictions and labels in a dictionary
    #return {'y': y, 'probs': probs, 'ypred': ypred, 'acc': acc}
    return y,probs,ypred,acc

def get_dls_metrics(dls,model,aug_pipelines_test,int_to_classes): #note that we can't call dls.vocab as it might be smaller on the test set
    "get metrics from model and dataloader"

    ytest,probs,preds,Acc = predict_whole_model(dls,model,aug_pipelines_test,numavg=3)
    metrics = classification_report_wrapper(preds, ytest,int_to_classes, print_report=True)
    
    plot_roc(ytest,probs,int_to_classes)
    auc_dict = Auc_Dict(ytest,probs,int_to_classes)
    print(f'auc_dict is: {auc_dict}')
    plot_pr(ytest,probs,int_to_classes)
    pr_dict = Pr_Dict(ytest,probs,int_to_classes)
    print(f'auc_dict is: {pr_dict}')

    metrics['ytest']=ytest
    metrics['probs']=probs
    metrics['preds']=preds
    metrics['acc']=Acc
    metrics['auc_dict']=auc_dict
    metrics['pr_dict']=pr_dict

    return metrics

def get_xval_metrics(xval,yval,model,aug_pipelines_test,int_to_classes,numavg=3): #note that we can't call dls.vocab as it might be smaller on the test set
    "get metrics from gives batch (xval,yval)"

    probs,preds,Acc = predict_model(xval,yval,model,aug_pipelines_test,numavg=3)
    metrics = classification_report_wrapper(preds, yval,int_to_classes, print_report=True)
    metrics['acc']=Acc

    return metrics

def Mean_Results(results,vocab):
    "Get mean classif report and display it"

    lst = list(vocab) + ['accuracy', 'macro avg', 'weighted avg']
    reports=[]
    accs=[]
    for i in results.keys():
        if type(i)!=int:
            continue
        report = {j:results[i][j] for j in results[i].keys() if j in lst}
        reports.append(report)
        accs.append(results[i]['acc'])
    mean_report = Mean_Report(reports,vocab)
    print(format_classification_report(mean_report))
    
    print(f'mean acc is {mean(accs)} with std {stdev(accs)}')

    return mean_report

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()